In [1]:
import pandas as pd 
import numpy as np
import re
import json
import requests
from bs4 import BeautifulSoup
import bar_chart_race as bcr

In [2]:
URL = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id=2012;type=year'
page = requests.get(URL)
bs = BeautifulSoup(page.content, 'lxml')
table_body=bs.find_all('tbody')
# df = pd.DataFrame(columns=['Winner','Date'])

In [3]:
li = []
rows = table_body[0].find_all('tr')
for row in rows:
    cols = row.find_all('td')
    li.append([cols[2].string, cols[5].string])
    #print(cols[2].string + ' ' + cols[5].string )

In [4]:
df = pd.DataFrame(li, columns = ['Winner', 'Date'])

In [5]:
df.head()

,Winner,Date
0,South Africa,"Jan 11, 2012"
1,South Africa,"Jan 14, 2012"
2,South Africa,"Jan 17, 2012"
3,Sri Lanka,"Jan 20, 2012"
4,Sri Lanka,"Jan 22, 2012"


In [6]:
def extract_match_results(year):
    #URL = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id=2012;type=year'
    URL = 'https://stats.espncricinfo.com/ci/engine/records/team/match_results.html?class=2;id=' + year + ';type=year'
    page = requests.get(URL)
    bs = BeautifulSoup(page.content, 'lxml')
    table_body=bs.find_all('tbody')
    li = []
    rows = table_body[0].find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        li.append([cols[2].string, cols[5].string])
    return li

In [7]:
li = extract_match_results('2013')

In [8]:
final_list = []
for year in range(1971,2022):
    li = extract_match_results(str(year))
    for elem in li:
        final_list.append(elem)

In [9]:
df = pd.DataFrame(final_list, columns = ['Team', 'Date'])

In [10]:
df.head(100)

,Team,Date
0,Australia,"Jan 5, 1971"
1,England,"Aug 24, 1972"
2,Australia,"Aug 26, 1972"
3,England,"Aug 28, 1972"
4,New Zealand,"Feb 11, 1973"
...,...,...
95,West Indies,"Dec 5, 1980"
96,India,"Dec 6, 1980"
97,Australia,"Dec 7, 1980"
98,India,"Dec 9, 1980"


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4338 entries, 0 to 4337
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Team    4338 non-null   object
 1   Date    4338 non-null   object
dtypes: object(2)
memory usage: 67.9+ KB


In [12]:
df['Date'] = df.Date.str.replace(',', '')

In [13]:
df['Date'].head(100)

0      Jan 5 1971
1     Aug 24 1972
2     Aug 26 1972
3     Aug 28 1972
4     Feb 11 1973
         ...     
95     Dec 5 1980
96     Dec 6 1980
97     Dec 7 1980
98     Dec 9 1980
99    Dec 18 1980
Name: Date, Length: 100, dtype: object

In [14]:
#df['Date'].astype('datetime64[ns]')

df['Date'] = pd.to_datetime(df['Date'],format='%b %d %Y', errors='coerce')

In [15]:
df.head()

,Team,Date
0,Australia,1971-01-05
1,England,1972-08-24
2,Australia,1972-08-26
3,England,1972-08-28
4,New Zealand,1973-02-11


In [16]:
df[df.Date.isnull()].count()

Team    43
Date     0
dtype: int64

In [17]:
df.dropna(inplace=True)

In [18]:
df.shape

(4295, 2)

In [19]:
#df.drop(columns='Year', inplace=True)

In [20]:
#df['Year'] = df['Date'].dt.strftime('%Y-%m')

df['Date'] = pd.to_datetime(df['Date'].dt.strftime('%Y'))

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4295 entries, 0 to 4337
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Team    4295 non-null   object        
 1   Date    4295 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(1)
memory usage: 100.7+ KB


In [22]:
df.head(10)

,Team,Date
0,Australia,1971-01-01
1,England,1972-01-01
2,Australia,1972-01-01
3,England,1972-01-01
4,New Zealand,1973-01-01
5,England,1973-01-01
6,no result,1973-01-01
7,England,1973-01-01
8,West Indies,1973-01-01
9,Australia,1974-01-01


In [23]:
#df.groupby(['Date','Team']).size().unstack(fill_value=0)

In [24]:
#pd.crosstab(df['Date'], df['Team'].fillna(0))

In [25]:
#df_data = pd.pivot_table(df, columns=['Team'], index=['Date'], aggfunc='size', fill_value=0)

In [26]:
df_cricket = df.groupby(['Date', 'Team']).size().groupby(level=1).cumsum().unstack(fill_value=0)

In [27]:
df_cricket.head()

Team,Afghanistan,Africa XI,Asia XI,Australia,Bangladesh,Bermuda,Canada,England,Hong Kong,ICC World XI,...,Pakistan,Scotland,South Africa,Sri Lanka,U.A.E.,U.S.A.,West Indies,Zimbabwe,no result,tied
Date,,,,,,,,,,,,,,,,,,,,,
1971-01-01,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1972-01-01,0,0,0,2,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1973-01-01,0,0,0,0,0,0,0,4,0,0,...,0,0,0,0,0,0,1,0,1,0
1974-01-01,0,0,0,4,0,0,0,5,0,0,...,2,0,0,0,0,0,0,0,0,0
1975-01-01,0,0,0,8,0,0,0,9,0,0,...,3,0,0,0,0,0,6,0,3,0


In [28]:
df_cricket = df_cricket.drop(columns=['no result', 'tied', 'U.A.E.', 'U.S.A.', 'Scotland', 'ICC World XI', 'Asia XI', 'Africa XI', 
                                'Bermuda', 'Canada', 'Nepal', 'Hong Kong', 'Kenya', 'Ireland', 'Namibia', 'Netherlands', 
                                'Oman', 'P.N.G.',])

In [29]:
df_cricket.head()

Team,Afghanistan,Australia,Bangladesh,England,India,New Zealand,Pakistan,South Africa,Sri Lanka,West Indies,Zimbabwe
Date,,,,,,,,,,,
1971-01-01,0,1,0,0,0,0,0,0,0,0,0
1972-01-01,0,2,0,2,0,0,0,0,0,0,0
1973-01-01,0,0,0,4,0,1,0,0,0,1,0
1974-01-01,0,4,0,5,0,0,2,0,0,0,0
1975-01-01,0,8,0,9,1,3,3,0,0,6,0


In [30]:
bcr.bar_chart_race(df=df_cricket, title = 'ODI Match Wins', filename='odi_wins.mp4')